In [1]:
import nltk
import os
import re
import sklearn
import pandas as pd
import numpy as np

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB,MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score,confusion_matrix

In [3]:
from sklearn.feature_selection import chi2,SelectKBest

In [4]:
from sklearn.ensemble import BaggingClassifier

In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/u4512/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/u4512/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [7]:
with open('Kitchentrain.txt','r',encoding='utf-8') as f:
    kitchen_train = f.readlines()

In [8]:
stopword = stopwords.words('english')

In [9]:
def preprocess(sentence):
    sentence = re.sub('[^\w\s]',' ',str(sentence))
    sentence = re.sub('[^a-zA-Z]',' ',str(sentence))
    new_sent = " "
    tok = word_tokenize(sentence)
    for i in range(len(tok)):
        if tok[i].lower() not in stopword:
            new_sent+=tok[i].lower()+" "
    return new_sent

In [10]:
corpus_train = []

In [11]:
label_train = np.zeros(1600)

In [12]:
label_train[0:800] = 1

In [13]:
for i in range(len(kitchen_train)):
    sent = kitchen_train[i]
    sent = sent[0:len(sent)-1]
    corpus_train.append(preprocess(sent))

In [14]:
len(corpus_train)

1600

In [15]:
len(label_train)

1600

In [16]:
with open('../Dvd/Dvdtest.txt','r',encoding='utf-8') as f:
    dvd_test = f.readlines()

In [17]:
corpus_test = []

In [18]:
label_test = np.zeros(400)

In [19]:
label_test[0:200] = 1

In [20]:
for i in range(400):
    sent = dvd_test[i]
    sent = sent[0:len(sent)-1]
    corpus_test.append(preprocess(sent))

In [21]:
len(corpus_test)

400

In [22]:
len(label_test)

400

In [23]:
train_length = len(corpus_train)
test_length = len(corpus_test)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
vectorizer = TfidfVectorizer(min_df = 3,max_df=0.8,use_idf=True,sublinear_tf=True,stop_words='english')

In [26]:
train_corpus_tf_idf = vectorizer.fit_transform(corpus_train)

/home/u4512/.conda/envs/nlp/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [27]:
test_corpus_tf_idf = vectorizer.transform(corpus_test)

/home/u4512/.conda/envs/nlp/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [28]:
train_corpus_tf_idf.shape

(1600, 3034)

In [29]:
vectorizer_chi2 = SelectKBest(chi2,k=2500)

In [30]:
chi_train_corpus_tf_idf = vectorizer_chi2.fit_transform(train_corpus_tf_idf,label_train)

/home/u4512/.conda/envs/nlp/lib/python3.5/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [31]:
chi_test_corpus_tf_idf = vectorizer_chi2.transform(test_corpus_tf_idf)

/home/u4512/.conda/envs/nlp/lib/python3.5/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [32]:
lr_classifier = LogisticRegression()
lr_classifier.fit(chi_train_corpus_tf_idf,label_train)
lr_pred = lr_classifier.predict(chi_test_corpus_tf_idf)
lr_acc = float((sum(lr_pred==label_test))/len(label_test))
lr_f1 = f1_score(lr_pred,label_test)
lr_cm = confusion_matrix(lr_pred,label_test)
print("The accuracy is :",lr_acc)
print("The f1 score is :",lr_f1)
print("confusion matrix is:")
print(lr_cm)

The accuracy is : 0.705
The f1 score is : 0.6810810810810811
confusion matrix is:
[[156  74]
 [ 44 126]]


In [44]:
#Bagging Classifier with Logistic Regression
bg_classifier_lr = BaggingClassifier(base_estimator=lr_classifier,n_estimators=2,random_state=0,max_samples=0.5)
bg_classifier_lr.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_lr = bg_classifier_lr.predict(chi_test_corpus_tf_idf)
bg_accuracy_lr = bg_classifier_lr.score(chi_test_corpus_tf_idf,label_test)
bg_f1 = f1_score(bg_predict_lr,label_test)
bg_cm_lr = confusion_matrix(label_test,bg_predict_lr)
print("The accuracy is : ",bg_accuracy_lr)
print("The f1 score is :",bg_f1)
print("The confusion matrix is:")
print(bg_cm_lr)

The accuracy is :  0.725
The f1 score is : 0.7135416666666667
The confusion matrix is:
[[153  47]
 [ 63 137]]


In [45]:
rbf_classifier = svm.SVC(gamma=0.9)
rbf_classifier.fit(chi_train_corpus_tf_idf,label_train)
rbf_pred = rbf_classifier.predict(chi_test_corpus_tf_idf)
rbf_accuracy = rbf_classifier.score(chi_test_corpus_tf_idf,label_test)
rbf_cm = confusion_matrix(rbf_pred,label_test)
rbf_f1 = f1_score(rbf_pred,label_test)
print("The SVM (rbf) Accuracy is: ",rbf_accuracy)
print("The F1 score is: ",rbf_f1)
print("The confusion matrix is:")
print(rbf_cm)

The SVM (rbf) Accuracy is:  0.7225
The F1 score is:  0.7055702917771882
The confusion matrix is:
[[156  67]
 [ 44 133]]


In [95]:
#Bagging Classifier SVM with RBF Kernel
bg_classifier_rbf = BaggingClassifier(base_estimator=rbf_classifier,n_estimators=8,random_state=0,max_samples=0.8)
bg_classifier_rbf.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_rbf = bg_classifier_rbf.predict(chi_test_corpus_tf_idf)
bg_accuracy_rbf = bg_classifier_rbf.score(chi_test_corpus_tf_idf,label_test)
bg_cm_rbf = confusion_matrix(label_test,bg_predict_rbf)
bg_f1_rbf = f1_score(label_test,bg_predict_rbf)
print("The accuracy is : ",bg_accuracy_rbf)
print("The F1 score is: ",bg_f1_rbf)
print("The confusion matrix is:")
print(bg_cm_rbf)

The accuracy is :  0.7025
The F1 score is:  0.6721763085399449
The confusion matrix is:
[[159  41]
 [ 78 122]]


In [84]:
dt_classifier = DecisionTreeClassifier(random_state=0,max_features=1.0)
dt_classifier.fit(chi_train_corpus_tf_idf,label_train)
dt_pred = dt_classifier.predict(chi_test_corpus_tf_idf)
dt_accuracy = dt_classifier.score(chi_test_corpus_tf_idf,label_test)
dt_cm = confusion_matrix(dt_pred,label_test)
dt_f1 = f1_score(label_test,dt_pred)
print("The DT Accuracy is: ",dt_accuracy)
print("The F1 score is :",dt_f1)
print("The confusion matrix is:")
print(dt_cm)

The DT Accuracy is:  0.635
The F1 score is : 0.6540284360189573
The confusion matrix is:
[[116  62]
 [ 84 138]]


In [88]:
#Bagging with Decision Tree
bg_classifier_dt = BaggingClassifier(base_estimator=dt_classifier,n_estimators=15,random_state=0,max_samples=0.9)
bg_classifier_dt.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_dt = bg_classifier_dt.predict(chi_test_corpus_tf_idf)
bg_accuracy_dt = bg_classifier_dt.score(chi_test_corpus_tf_idf,label_test)
bg_cm_dt = confusion_matrix(label_test,bg_predict_dt)
bg_f1_dt = f1_score(label_test,bg_predict_dt)
print("The accuracy is: ",bg_accuracy_dt)
print("The F1 score is: ",bg_f1_dt)
print("The confusion matrix is:")
print(bg_cm_dt)

The accuracy is:  0.66
The F1 score is:  0.6512820512820513
The confusion matrix is:
[[137  63]
 [ 73 127]]


In [65]:
linear_classifier = svm.SVC(kernel='linear')
linear_classifier.fit(chi_train_corpus_tf_idf,label_train)
linear_pred = linear_classifier.predict(chi_test_corpus_tf_idf)
linear_accuracy = linear_classifier.score(chi_test_corpus_tf_idf,label_test)
linear_cm = confusion_matrix(linear_pred,label_test)
linear_f1 = f1_score(label_test,linear_pred)
print("The SVM(linear) Accuracy is: ",linear_accuracy)
print("The F1 score is :",linear_f1)
print("The confusion matrix is:")
print(linear_cm)

The SVM(linear) Accuracy is:  0.705
The F1 score is : 0.6927083333333334
The confusion matrix is:
[[149  67]
 [ 51 133]]


In [73]:
#Bagging Classifier SVM with Linear Kernel
bg_classifier_linear = BaggingClassifier(base_estimator=linear_classifier,n_estimators=5,random_state=0,max_samples=0.9)
bg_classifier_linear.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_linear = bg_classifier_linear.predict(chi_test_corpus_tf_idf)
bg_accuracy_linear = bg_classifier_linear.score(chi_test_corpus_tf_idf,label_test)
bg_cm_linear = confusion_matrix(label_test,bg_predict_linear)
bg_f1_linear = f1_score(bg_predict_linear,label_test)
print("The accuracy is : ",bg_accuracy_linear)
print("The F1 score is :",bg_f1_linear)
print("The confusion matrix is:")
print(bg_cm_linear)

The accuracy is :  0.7125
The F1 score is : 0.6997389033942559
The confusion matrix is:
[[151  49]
 [ 66 134]]


In [74]:
nb_classifier = MultinomialNB()
nb_classifier.fit(chi_train_corpus_tf_idf,label_train)
nb_acc = nb_classifier.score(chi_test_corpus_tf_idf,label_test)
nb_pred = nb_classifier.predict(chi_test_corpus_tf_idf)
nb_cm = confusion_matrix(nb_pred,label_test)
nb_f1 = f1_score(nb_pred,label_test)
print("Accuracy is :",nb_acc)
print("F1 score is :",nb_f1)
print("The confusion matrix is:")
print(nb_cm)

Accuracy is : 0.7275
F1 score is : 0.7029972752043597
The confusion matrix is:
[[162  71]
 [ 38 129]]


In [75]:
#Bagging with Naive Bayes
bg_classifier_nb = BaggingClassifier(base_estimator=nb_classifier,n_estimators=3,random_state=0,max_samples=0.9)
bg_classifier_nb.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_nb = bg_classifier_nb.predict(chi_test_corpus_tf_idf)
bg_accuracy_nb = bg_classifier_nb.score(chi_test_corpus_tf_idf,label_test)
bg_cm_nb = confusion_matrix(label_test,bg_predict_nb)
bg_f1_nb = f1_score(bg_predict_nb,label_test)
print("The accuracy is: ",bg_accuracy_nb)
print("The F1 score is: ",bg_f1_nb)
print("The confusion matrix is:")
print(bg_cm_nb)

The accuracy is:  0.7325
The F1 score is:  0.7304785894206549
The confusion matrix is:
[[148  52]
 [ 55 145]]


In [76]:
knn_classifier = KNeighborsClassifier(n_neighbors=7)
knn_classifier.fit(chi_train_corpus_tf_idf,label_train)
knn_pred = knn_classifier.predict(chi_test_corpus_tf_idf)
knn_accuracy = knn_classifier.score(chi_test_corpus_tf_idf,label_test)
knn_cm = confusion_matrix(knn_pred,label_test)
knn_f1 = f1_score(knn_pred,label_test)
print("Accuracy is :",knn_accuracy)
print("F1 score is :",knn_f1)
print("The confusion matrix is:")
print(knn_cm)

Accuracy is : 0.53
F1 score is : 0.5965665236051503
The confusion matrix is:
[[ 73  61]
 [127 139]]


In [77]:
#Bagging with KNN
bg_classifier_knn = BaggingClassifier(base_estimator=knn_classifier,n_estimators=10,random_state=0,max_samples=0.8)
bg_classifier_knn.fit(chi_train_corpus_tf_idf,label_train)
bg_predict_knn = bg_classifier_knn.predict(chi_test_corpus_tf_idf)
bg_accuracy_knn = bg_classifier_knn.score(chi_test_corpus_tf_idf,label_test)
bg_cm_knn = confusion_matrix(label_test,bg_predict_knn)
bg_f1_knn = f1_score(bg_predict_knn,label_test)
print("The accuracy is: ",bg_accuracy_knn)
print("The F1 score is: ",bg_f1_knn)
print("The confusion matrix is:")
print(bg_cm_knn)

The accuracy is:  0.5425
The F1 score is:  0.6678765880217785
The confusion matrix is:
[[ 33 167]
 [ 16 184]]
